# <b style="color:rgb(247, 118, 167)">Runnable</b>

In [1]:
import sys
sys.path.append('../..')

from ck_helper import *
ready_langchain('CVAL')

LangSmith 추적을 시작합니다.
[프로젝트명]
CVAL


In [3]:
model_name = 'gpt-4o-mini'

#### **RunnablePassthourgh**

In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [11]:
prompt = PromptTemplate.from_template('{symtom}의 원인은?')
model = ChatOpenAI(model=model_name, max_tokens=150)
chain = prompt | model
chain.invoke({'symtom':'어깨통증'})

AIMessage(content='어깨 통증의 원인은 여러 가지가 있을 수 있습니다. 일반적인 원인은 다음과 같습니다:\n\n1. **근육 및 인대 손상**: 과도한 사용, 염좌, 또는 찢어짐 등으로 인해 어깨의 근육이나 인대가 손상될 수 있습니다.\n\n2. **회전근개 손상**: 어깨의 회전근개는 여러 개의 근육과 힘줄로 이루어져 있으며, 이 부분에 염증이나 손상이 생기면 통증이 발생할 수 있습니다. 이는 일반적으로 스포츠 활동이나 반복적인 동작으로 인해 발생합니다.\n\n3. **관절염**: 어깨 관절의 염증', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 150, 'prompt_tokens': 15, 'total_tokens': 165, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b376dfbbd5', 'finish_reason': 'length', 'logprobs': None}, id='run-50462120-4ffa-4278-b603-3c069864c1a8-0', usage_metadata={'input_tokens': 15, 'output_tokens': 150, 'total_tokens': 165, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [6]:
from langchain_core.runnables import RunnablePassthrough

In [8]:
runnable = RunnablePassthrough().invoke({'symtom':'어깨통증'})

In [9]:
type(runnable)

dict

In [12]:
runnable_chain = {'symtom': RunnablePassthrough()} | prompt | model
result = runnable_chain.invoke({'symtom':'어깨통증'})
print(result.content)

어깨 통증의 원인은 다양할 수 있으며, 주요 원인으로는 다음과 같은 것들이 있습니다:

1. **견관절염**: 나이가 들면서 관절의 연골이 퇴화하여 발생하는 염증.
2. **회전근개 부상**: 어깨의 회전근개가 손상되거나 염증이 생기는 경우.
3. **어깨 충돌 증후군**: 어깨의 힘줄이나 버섯 모양의 구조물(봉우리) 사이에 충돌이 발생하여 통증을 유발.
4. **어깨 탈구**: 어깨 관절이 제자리에서 빠지거나 탈구되는


In [20]:
RunnablePassthrough().assign(new_value=lambda x: x['symtom'][:50]).invoke({'symtom':'어깨통증'})

{'symtom': '어깨통증', 'new_value': '어깨통증'}

#### <b>RunnableParallel - <span style="color:#A05F09;">Input 값을 변경하는데 사용되는가 보다</span></b>


In [21]:
from langchain_core.runnables import RunnableParallel

In [22]:
runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    extra=RunnablePassthrough.assign(mult=lambda x: x['num'] * 3),
    modified=lambda x: x['num'] + 1,
)

In [23]:
runnable.invoke({'num': 1})

{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

#### <b>RunnableLambda</b>


In [25]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from datetime import datetime

In [26]:
def get_today(a):
    return datetime.today().strftime('%b-%d')

get_today(None)

'Apr-01'

In [27]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [28]:
prompt = PromptTemplate.from_template(
    '{today}가 생일인 유명인 {n}명을 나열하고 생년월일을 표기하라'
)
llm = ChatOpenAI(model=model_name, max_tokens=150)

chain = (
    {'today': RunnableLambda(get_today), 'n': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [29]:
result = chain.invoke(3)
print(result)

다음은 4월 1일에 태어난 유명인 3명과 그들의 생년월일입니다:

1. **파블로 피카소 (Pablo Picasso)** - 1881년 4월 1일
2. **리처드 도킨스 (Richard Dawkins)** - 1941년 4월 1일
3. **샤일로 핀 (Sharon Stone)** - 1958년 4월 1일

이 외에도 4월 1일에 태어난 많은 유명인들이 있습니다!


In [30]:
from operator import itemgetter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI

In [33]:
def length_function(text):
    return len(text)

def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)

def multiple_length_function(_dict):
    return _multiple_length_function(_dict['text1'], _dict['text2'])

prompt = ChatPromptTemplate.from_template('{a} + {b}는?')
llm = ChatOpenAI(model=model_name, max_tokens=150)

chain1 = prompt | model
chain = ({
    'a': itemgetter('word1') | RunnableLambda(length_function),
    'b': {'text1': itemgetter('word1'), 'text2': itemgetter('word2')} | RunnableLambda(multiple_length_function)
} | chain1)

In [34]:
chain.invoke({'word1': 'hello', 'word2': 'world'})

AIMessage(content='5 + 25는 30입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 13, 'total_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b376dfbbd5', 'finish_reason': 'stop', 'logprobs': None}, id='run-9b507aee-92bf-47d9-8b0e-8d092e8e22bf-0', usage_metadata={'input_tokens': 13, 'output_tokens': 10, 'total_tokens': 23, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

---

In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [3]:
model_name = 'gpt-4o-mini'

In [4]:
prompt_template = PromptTemplate.from_template('{food}의 단백질 양은?')
llm = ChatOpenAI(model=model_name, temperature=0)
chain = prompt_template| llm

In [5]:
response = chain.invoke({'food': '두부'})
print(response)

content='두부의 단백질 양은 제품의 종류와 제조 방법에 따라 다를 수 있지만, 일반적으로 100g의 두부에는 약 8g에서 12g 정도의 단백질이 포함되어 있습니다. 연두부(부드러운 두부)는 상대적으로 단백질 함량이 낮고, 단단한 두부는 더 높은 단백질 함량을 가질 수 있습니다. 두부는 식물성 단백질의 좋은 공급원으로, 채식주의자나 비건 식단에서 자주 사용됩니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 119, 'prompt_tokens': 16, 'total_tokens': 135, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None} id='run-1c2fb18e-ec71-4aa5-b160-e3cbbecc4b2c-0' usage_metadata={'input_tokens': 16, 'output_tokens': 119, 'total_tokens': 135, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [ ]:
from langchain_core.runnables import RunnablePassthouth